In [1]:
#%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from loaddata import load_mnist
import model

In [2]:
data = load_mnist()
tf.reset_default_graph()
n_epochs  = 100
n_sgd     = 128
n_eta_sgd = 1000
beta      = 0.05
beta_epoch = 10
n = model.Net(512, trainable_sigma=False, log_sigma2=0, init_beta=0.0)

# train model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    n_mini_batches = int(len(data['train_labels']) / n_sgd)

    for epoch in range(n_epochs):
        # randomize order of training data
        permutation = np.random.permutation(len(data['train_labels']))
        train_data = data['train_data'][permutation]
        train_labels = data['train_labels'][permutation]
        
        x_batch = train_data[:n_eta_sgd]
        y_batch = train_labels[:n_eta_sgd]
        
        if epoch == 0:
            n.eta_optimizer.minimize(sess, feed_dict={n.x: x_batch})
        if n.trainable_sigma:
            n.sigma_optimizer.minimize(sess, feed_dict={n.x: x_batch, n.y: y_batch})

                        
        for batch in range(n_mini_batches):
            # sample mini-batch
            x_batch = train_data[batch * n_sgd:(1 + batch) * n_sgd]
            y_batch = train_labels[batch * n_sgd:(1 + batch) * n_sgd]

            cparams = {n.x: x_batch, n.y: y_batch}
            sess.run(n.trainstep, feed_dict=cparams)
            
        if epoch == beta_epoch:
            assign_op = n.beta.assign(beta)
            sess.run(assign_op)

        if True:
            x_batch = data['test_data'][::10]
            y_batch = data['test_labels'][::10]
            cparams = {n.x: x_batch, n.y: y_batch}

            acc  = sess.run(n.accuracy, feed_dict=cparams)
            loss = sess.run(n.loss, feed_dict=cparams)
            Ixt  = sess.run(n.Ixt, feed_dict=cparams)
            Iyt  = sess.run(n.Iyt, feed_dict=cparams)
            
            
            print()
            print('epoch             :', epoch+1, '/', n_epochs)
            print('accuracy          :', acc)
            print('loss              :', loss)
            print('mutual info I(X;T):', Ixt)
            print('mutual info I(Y;T):', Iyt)
            print('beta:             :', sess.run(n.beta))
            #print('learning rate     :', lr)
            #tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_ph, logits=self.decoder()))
            print('noise variance    :', sess.run(tf.exp(n.log_sigma2)))
            print('kernel width      :', sess.run(tf.exp(n.log_eta2)))
            #print('time              :', time.time() - start)

            


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


epoch             : 1 / 100
accuracy          : 0.927
loss              : -2.0797834
mutual info I(X;T): 16.973633
mutual info I(Y;T): 2.0792518
beta:             : 0.0
noise variance    : 1.0
kernel width      : 0.04010434

epoch             : 2 / 100
accuracy          : 0.961
loss              : -2.1621115
mutual info I(X;T): 16.974121
mutual info I(Y;T): 2.1755934
beta:             : 0.0
noise variance    : 1.0
kernel width      : 0.04010434

epoch             : 3 / 100
accuracy          : 0.96
loss              : -2.1722233
mutual info I(X;T): 16.974243
mutual info I(Y;T): 2.1759157
beta:             : 0.0
noise variance    : 1.0
kernel width      : 0.04010434

epoch             : 4 / 100
accuracy          : 0.967
loss              : -2.1901848
mutu


epoch             : 36 / 100
accuracy          : 0.978
loss              : 0.24269915
mutual info I(X;T): 6.9415283
mutual info I(Y;T): 2.1597304
beta:             : 0.05
noise variance    : 1.0
kernel width      : 0.00013065226

epoch             : 37 / 100
accuracy          : 0.977
loss              : 0.23400164
mutual info I(X;T): 6.935608
mutual info I(Y;T): 2.1429787
beta:             : 0.05
noise variance    : 1.0
kernel width      : 0.00010782827

epoch             : 38 / 100
accuracy          : 0.962
loss              : 0.30067563
mutual info I(X;T): 6.930786
mutual info I(Y;T): 2.0983284
beta:             : 0.05
noise variance    : 1.0
kernel width      : 8.8959045e-05

epoch             : 39 / 100
accuracy          : 0.983
loss              : 0.20752358
mutual info I(X;T): 6.926758
mutual info I(Y;T): 2.1898015
beta:             : 0.05
noise variance    : 1.0
kernel width      : 7.3369556e-05

epoch             : 40 / 100
accuracy          : 0.975
loss              : 0.22501


epoch             : 72 / 100
accuracy          : 0.971
loss              : 0.2648065
mutual info I(X;T): 6.907898
mutual info I(Y;T): 2.1149173
beta:             : 0.05
noise variance    : 1.0
kernel width      : 1.2188498e-07

epoch             : 73 / 100
accuracy          : 0.978
loss              : 0.2474339
mutual info I(X;T): 6.90802
mutual info I(Y;T): 2.1404257
beta:             : 0.05
noise variance    : 1.0
kernel width      : 1.0040291e-07

epoch             : 74 / 100
accuracy          : 0.977
loss              : 0.27368927
mutual info I(X;T): 6.908081
mutual info I(Y;T): 2.094696
beta:             : 0.05
noise variance    : 1.0
kernel width      : 8.272223e-08

epoch             : 75 / 100
accuracy          : 0.978
loss              : 0.29481435
mutual info I(X;T): 6.90802
mutual info I(Y;T): 2.0790603
beta:             : 0.05
noise variance    : 1.0
kernel width      : 6.814078e-08

epoch             : 76 / 100
accuracy          : 0.974
loss              : 0.26297784
mutu